In [3]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1747064939379_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1747064939379_0002,pyspark,idle,Link,Link,None,✔


In [4]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("boto3")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt1/yarn/usercache/livy/appcache/application_1747064939379_0002/container_1747064939379_0002_01_000001/tmp/spark-428f6e0e-7421-4b44-9516-c3efd3e55bb4
    Can't uninstall 'python-dateutil'. No files were found to uninstall.




ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.23.11 requires python-dateutil<=2.9.0,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.




In [5]:
import pandas as pd
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import count
import matplotlib.pyplot as plt


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark = SparkSession.builder.appName("Data_prep").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Load ratings
movies = spark.read.csv("s3://recomendandoando/movies.csv", header=True, inferSchema=True)
ratings= spark.read.csv("s3://recomendandoando/ratings.csv", header=True, inferSchema=True)

ratings.show(5)
movies.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|     17|   4.0|944249077|
|     1|     25|   1.0|944250228|
|     1|     29|   2.0|943230976|
|     1|     30|   5.0|944249077|
|     1|     32|   5.0|943228858|
+------+-------+------+---------+
only showing top 5 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
ratings.printSchema()
ratings.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|          32000204|          32000204|          32000204|            32000204|
|   mean|100278.50641102163|29318.610121829224|3.5403956487277393|1.2752411995732634E9|
| stddev|57949.046233252906| 50958.16087967011| 1.058986213945308|2.5616297595059687E8|
|    min|                 1|                 1|               0.5|           789652004|
|    max|            200948|            292757|               5.0|          1697164147|
+-------+------------------+------------------+------------------+--------------------+

In [9]:
num_rows = movies.count()
num_cols = len(movies.columns)
print(f'movies.csv dimensions: {(num_rows, num_cols)}')
num_rows_rat = ratings.count()
num_cols_rat = len(ratings.columns)
print(f'ratings.csv dimensions: {(num_rows_rat, num_cols_rat)}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

movies.csv dimensions: (87585, 3)
ratings.csv dimensions: (32000204, 4)

In [10]:
# Delete duplicated movies
# Group by movie title and count occurrences
duplicate_movies = movies.groupBy("title").agg(count("*").alias("count"))

# Filter for movies with a count greater than 1 (duplicates)
duplicate_movies = duplicate_movies.filter("count > 1")

# Show the duplicate movies
duplicate_movies.show()
duplicate_movies_num = duplicate_movies.count()
duplicate_titles = (
    movies.groupBy("title")
    .agg(count("*").alias("count"))
    .filter("count > 1")
)
# Step 2: Join to get the movieIds of duplicated titles
duplicate_movie_ids = movies.join(duplicate_titles, on="title", how="inner") \
                            .select("movieId", "title")
duplicate_movie_ids.show()
# Step 3: Remove duplicates from ratings
ratings_filtered = ratings.join(duplicate_movie_ids, on="movieId", how="left_anti")
# Step 4: Remove all movies whose title appeared more than once
movies_filtered = movies.join(
    duplicate_movie_ids.select("movieId"),  # the IDs of all duplicated titles
    on="movieId",
    how="left_anti"                         # keep only those not in duplicate_movie_ids
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|               title|count|
+--------------------+-----+
|      Dashcam (2021)|    2|
|         Eros (2004)|    2|
|Let There Be Ligh...|    2|
|         Hide (2011)|    2|
|     Paradise (2013)|    2|
|  The Promise (2016)|    2|
|       Junior (2011)|    2|
|Marvel Super Hero...|    2|
|      The Box (2003)|    2|
|          Ava (2017)|    2|
|         Ride (2018)|    2|
|        Beast (2022)|    2|
|The Connection (2...|    2|
|     Deranged (2012)|    2|
|       Office (2015)|    2|
|The Lonely Island...|    2|
|The Harbinger (2022)|    2|
|   The Plague (2006)|    3|
|Beauty and the Be...|    2|
|   Summerland (2020)|    2|
+--------------------+-----+
only showing top 20 rows

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|    588|      Aladdin (1992)|
|   1788|Men with Guns (1997)|
|   2644|      Dracula (1931)|
|   2851|     Saturn 3 (1980)|
|   3553|       Gossip (2000)|
|   3598|       Hamlet (2000)|

In [11]:
num_rows = ratings_filtered.count()
num_cols = len(ratings_filtered.columns)
print(f"Cleaned ratings.csv dimensions: {(num_rows, num_cols)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cleaned ratings.csv dimensions: (31901794, 4)

In [12]:
duplicate_counts = ratings_filtered.groupBy("userId", "movieId") \
                         .count() \
                         .filter("count > 1")

# Show duplicates (if any)
duplicate_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-----+
|userId|movieId|count|
+------+-------+-----+
+------+-------+-----+

In [13]:
ratings_filtered.filter((ratings_filtered.rating < 0.5) | (ratings_filtered.rating > 5)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+---------+
|movieId|userId|rating|timestamp|
+-------+------+------+---------+
+-------+------+------+---------+

## Store the processed dataframes

In [15]:
# write ratings
ratings_filtered.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("s3://recomendandoando/processed/ratings_filtered/")

# write movies
movies_filtered.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header",    "true") \
    .option("quoteAll",  "true") \
    .option("escape",    "\"") \
    .csv("s3://recomendandoando/processed/movies_filtered/")


# 1) Imports & client
import boto3
bucket = "recomendandoando"
s3 = boto3.client("s3")

# 2) Helper to copy shard → new folder
def copy_shard_to(folder, src_prefix, dest_prefix, dest_filename):
    """
    folder: your S3 bucket name
    src_prefix: existing folder with part-*.csv shards (must end with '/')
    dest_prefix: NEW folder where you want your single CSV (must end with '/')
    dest_filename: the name you want in the new folder (e.g. "ratings_filtered.csv")
    """
    # list the shard
    resp = s3.list_objects_v2(Bucket=bucket, Prefix=src_prefix)
    source_key = next(
        (o["Key"] for o in resp.get("Contents", [])
         if o["Key"].endswith(".csv") and "part-00000" in o["Key"]),
        None
    )
    if source_key is None:
        raise RuntimeError(f"No part-00000 shard found under {src_prefix}")
    # define the destination key
    dest_key = f"{dest_prefix}{dest_filename}"
    # copy
    s3.copy_object(
        Bucket=bucket,
        CopySource={"Bucket": bucket, "Key": source_key},
        Key=dest_key
    )
    print(f"Copied {source_key} → {dest_key}")

# 3) Use it for ratings and movies, pointing at a new folder:
#    Here I choose "processed/clean/" as the target folder — you can rename it as you like.

copy_shard_to(
    bucket,
    src_prefix="processed/ratings_filtered/",
    dest_prefix="processed/clean/ratings/",
    dest_filename="ratings_filtered.csv"
)

copy_shard_to(
    bucket,
    src_prefix="processed/movies_filtered/",
    dest_prefix="processed/clean/movies/",
    dest_filename="movies_filtered.csv"
)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Copied processed/ratings_filtered/part-00000-efea96bf-acf8-4725-94b8-ed8216e5dd68-c000.csv ? processed/clean/ratings/ratings_filtered.csv
Copied processed/movies_filtered/part-00000-567b01c5-d724-4576-a643-10c5cdf991cc-c000.csv ? processed/clean/movies/movies_filtered.csv